In [6]:
import tensorflow as tf
import numpy as np
import os
import _Data
import _Model 
import _Metric
import time
import random
import imp
from _utils import u_constant
path = u_constant.PATH_ROOT + "for learn/Python/RNN-for-Joint-NLU/"
train_data_path = path + "dataset/atis-2.train.w-intent.iob"
test_data_path = path + "dataset/atis-2.dev.w-intent.iob"
model_path = path + "model/"

data_info = _Data.DataUtils(train_data_path, test_data_path, length=50)
indexed_train_data  = data_info.to_index(data_info.train_data)  # 4478
indexed_test_data = data_info.to_index(data_info.test_data)  # 500

input_length = data_info.length  # 50
embedding_size = 64
hidden_size = 100
n_layers = 2
batch_size = 16
vocab_size = len(data_info.word2index)  # 871
slot_size = len(data_info.tag2index)  # 124
intent_size = len(data_info.intent2index)  # 22
epoch_num = 60

In [18]:
imp.reload(_Data)
imp.reload(_Metric)
imp.reload(_Model)

<module '_Model' from 'C:\\Users\\Cigar\\Documents\\jupyter\\NLP_learn\\RNN_for_Joint_NLU\\_Model.py'>

In [3]:
def get_iternum_from_ckpt(ckpt_path):
    iters = ckpt_path.split(".ckpt-")[1]
    return int(iters)

In [20]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 程序最多只能占用指定gpu80%的显存
config.gpu_options.allow_growth = True      #程序按需申请内存

model = _Model.Model(input_length, embedding_size, hidden_size, vocab_size, 
                     slot_size, intent_size, batch_size)
with tf.Session(graph=model.graph, config=config) as sess:
    ckpt = tf.train.get_checkpoint_state(model_path)
    if ckpt is not None:
        ckpt_path = ckpt.model_checkpoint_path
        model.saver.restore(sess, model_path)
        iteration = get_iternum_from_ckpt(ckpt_path)
    else:
        sess.run(tf.global_variables_initializer())
        iteration = 1
    start = time.time()
    avg_loss = 0.0
    for epoch in range(epoch_num):
        for batch in data_info.get_batch(batch_size, indexed_train_data):
            # 执行一个 batch 的训练
            _, loss, decoder_prediction, intent, mask, slot_w = model.step(sess, "train", batch)
            avg_loss += loss
            if iteration % 100 == 0:
                end = time.time()
                print("Iteration: %d " % iteration, 
                      "Avg. Train_loss: %.4f" % (avg_loss / 100),
                      "%.4f sec / batch" % ((end - start) / 100))
                start = time.time()
                avg_loss = 0.0
            iteration += 1
        # 每个 epoch保存一次模型
        model.saver.save(sess, model_path + "/seq2seq.ckpt", global_step=iteration)
        
        # 测试
        pred_slots = []
        true_slots = []
        slot_accs = []
        intent_accs = []
        for j, batch in enumerate(data_info.get_batch(batch_size, indexed_test_data)):
            decoder_prediction, intent = model.step(sess, "test", batch)
            decoder_prediction = np.transpose(decoder_prediction, [1, 0])
        
            test_in, test_true_length, test_out, test_intent = map(np.asarray, list(zip(*batch)))
            
            if j == 0:
                # 随机选出一个样本打印
                index = random.choice(range(len(batch)))
                print("---Epoch Summary for epoch %d---" % (epoch + 1))
                print("Input Sentence   :",\
                      data_info.recover_indexed_seq(test_in[index], "word")[: test_true_length[index]])
                print("Slot Truth       :",\
                      data_info.recover_indexed_seq(test_out[index], "slot")[: test_true_length[index]])
                print("Slot Pred        :",\
                      data_info.recover_indexed_seq(decoder_prediction[index], "slot")[: test_true_length[index]])
                print("Intent Truth     :", data_info.index2intent[test_intent[index]])
                print("Intent Pred      :", data_info.index2intent[intent[index]])
        
            slot_pred_length = np.shape(decoder_prediction)[1]
        
            # 确保输出与输入长度一致，便于指标计算
            # pad_width: pad element num for each axis
            # ((before_0, after_0), (before_1, after_1), ..., (before_N, after_N))
            pred_padded = np.lib.pad(decoder_prediction, 
                                     pad_width=((0, 0), (0, input_length - slot_pred_length)), 
                                     mode="constant", 
                                     constant_values=0)
            
            pred_slots.append(pred_padded)
            true_slots.append(test_out)
            
            slot_acc = _Metric.acc_for_sequence_batch(test_out, pred_padded, padding_token=0)
            intent_acc = _Metric._acc(test_intent, intent)
            
            slot_accs.append(slot_acc)
            intent_accs.append(intent_acc)
        
        pred_slots = np.vstack(pred_slots)
        true_slots = np.vstack(true_slots)
        test_slot_f1 = _Metric.f1_for_sequence_batch(true_slots, pred_slots, padding_token=0)
        
        print("Test Slot Avg.Acc   : %.4f" % np.average(slot_accs))
        print("Test Slot f1        : %.4f" % test_slot_f1)
        print("Test Intent Avg. Acc: %.4f" % np.average(intent_accs))
        

encoder outputs:  Tensor("concat:0", shape=(50, 16, 200), dtype=float32)
encoder outputs[0]:  Tensor("strided_slice:0", shape=(16, 200), dtype=float32)
encoder final state c:  Tensor("concat_1:0", shape=(16, 200), dtype=float32)
outputs:  BasicDecoderOutput(rnn_output=<tf.Tensor 'decode/decoder/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 16, 124) dtype=float32>, sample_id=<tf.Tensor 'decode/decoder/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, 16) dtype=int32>)
outputs.rnn_output:  Tensor("decode/decoder/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, 16, 124), dtype=float32)
outputs.sample_id:  Tensor("decode/decoder/TensorArrayStack_1/TensorArrayGatherV3:0", shape=(?, 16), dtype=int32)
decoder targets true length:  Tensor("strided_slice_1:0", shape=(?, 16), dtype=int32)
vars for loss function:  (<tf.Variable 'embedding:0' shape=(871, 64) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(164, 400) dtype=float32_ref>, <tf.Variable 'bidirec

Test Slot Avg.Acc   : 0.9497
Test Slot f1        : 0.9496
Test Intent Avg. Acc: 0.9657
Iteration: 2000  Avg. Train_loss: 0.1597 0.2554 sec / batch
Iteration: 2100  Avg. Train_loss: 0.1277 0.2276 sec / batch
Iteration: 2200  Avg. Train_loss: 0.1289 0.2260 sec / batch
---Epoch Summary for epoch 8---
Input Sentence   : ['i', 'want', 'a', 'flight', 'on', 'continental', 'from', 'boston', 'to', 'san', 'francisco']
Slot Truth       : ['O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name']
Slot Pred        : ['O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name']
Intent Truth     : atis_flight
Intent Pred      : atis_flight
Test Slot Avg.Acc   : 0.9598
Test Slot f1        : 0.9597
Test Intent Avg. Acc: 0.9677
Iteration: 2300  Avg. Train_loss: 0.1052 0.2371 sec / batch
Iteration: 2400  Avg. Train_loss: 0.0937 0.1785 sec / batch
Iteration: 2500  Avg. Train_loss: 0.0983 0.

Test Slot Avg.Acc   : 0.9757
Test Slot f1        : 0.9752
Test Intent Avg. Acc: 0.9657
Iteration: 5100  Avg. Train_loss: 0.0310 0.2338 sec / batch
Iteration: 5200  Avg. Train_loss: 0.0277 0.2231 sec / batch
Iteration: 5300  Avg. Train_loss: 0.0501 0.2111 sec / batch
---Epoch Summary for epoch 19---
Input Sentence   : ['what', 'are', 'the', 'flights', 'from', 'milwaukee', 'to', 'orlando', 'on', 'wednesday', 'night']
Slot Truth       : ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-depart_date.day_name', 'B-depart_time.period_of_day']
Slot Pred        : ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-depart_date.day_name', 'B-depart_time.period_of_day']
Intent Truth     : atis_flight
Intent Pred      : atis_flight
Test Slot Avg.Acc   : 0.9683
Test Slot f1        : 0.9684
Test Intent Avg. Acc: 0.9677
Iteration: 5400  Avg. Train_loss: 0.0511 0.2442 sec / batch
Iteration: 5500  Avg. Train_loss: 0.0406 0.2114 sec / batch


Test Slot Avg.Acc   : 0.9795
Test Slot f1        : 0.9794
Test Intent Avg. Acc: 0.9738
Iteration: 8400  Avg. Train_loss: 0.0293 0.2066 sec / batch
Iteration: 8500  Avg. Train_loss: 0.0427 0.1929 sec / batch
Iteration: 8600  Avg. Train_loss: 0.0347 0.1886 sec / batch
---Epoch Summary for epoch 31---
Input Sentence   : ['all', 'flights', 'from', 'pittsburgh', 'to', 'dallas', 'round', 'trip', 'after', '12', 'pm', 'less', 'than', '100']
Slot Truth       : ['O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'B-round_trip', 'I-round_trip', 'B-depart_time.time_relative', 'B-depart_time.time', 'I-depart_time.time', 'B-cost_relative', 'O', 'B-fare_amount']
Slot Pred        : ['O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'B-round_trip', 'I-round_trip', 'B-depart_time.time_relative', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'B-depart_date.day_number']
Intent Truth     : atis_flight
Intent Pred      : atis_flight
Test Slot Avg.Acc   : 0.9774
Test Slot f

---Epoch Summary for epoch 42---
Input Sentence   : ['on', 'thursday', "i'd", 'like', 'a', 'flight', 'from', 'st.', 'petersburg', 'to', 'miami']
Slot Truth       : ['O', 'B-depart_date.day_name', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'I-fromloc.city_name', 'O', 'B-toloc.city_name']
Slot Pred        : ['O', 'B-depart_date.day_name', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'I-fromloc.city_name', 'O', 'B-toloc.city_name']
Intent Truth     : atis_flight
Intent Pred      : atis_flight
Test Slot Avg.Acc   : 0.9824
Test Slot f1        : 0.9824
Test Intent Avg. Acc: 0.9657
Iteration: 11800  Avg. Train_loss: 0.0080 0.2583 sec / batch
Iteration: 11900  Avg. Train_loss: 0.0108 0.2008 sec / batch
---Epoch Summary for epoch 43---
Input Sentence   : ['list', 'all', 'american', 'airlines', 'from', 'milwaukee', 'to', 'phoenix', 'on', 'saturday']
Slot Truth       : ['O', 'O', 'B-airline_name', 'I-airline_name', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-depart_date.

Test Slot Avg.Acc   : 0.9843
Test Slot f1        : 0.9842
Test Intent Avg. Acc: 0.9657
Iteration: 14600  Avg. Train_loss: 0.0041 0.2130 sec / batch
Iteration: 14700  Avg. Train_loss: 0.0058 0.1749 sec / batch
---Epoch Summary for epoch 53---
Input Sentence   : ['what', 'is', 'the', 'fare', 'from', 'atlanta', 'to', 'boston', 'on', 'coach', 'one', 'way']
Slot Truth       : ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-class_type', 'B-round_trip', 'I-round_trip']
Slot Pred        : ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-class_type', 'B-round_trip', 'I-round_trip']
Intent Truth     : atis_airfare
Intent Pred      : atis_airfare
Test Slot Avg.Acc   : 0.9820
Test Slot f1        : 0.9817
Test Intent Avg. Acc: 0.9718
Iteration: 14800  Avg. Train_loss: 0.0070 0.2263 sec / batch
Iteration: 14900  Avg. Train_loss: 0.0038 0.1933 sec / batch
Iteration: 15000  Avg. Train_loss: 0.0063 0.2031 sec / batch
---Epoch Summary 